In [1]:
#!conda install requests
#!conda install beautifulsoup4
#!conda install selenium

In [2]:
import requests
from bs4 import BeautifulSoup
import selenium.webdriver
import pandas as pd

<b>Acknowledgements:</b> The code below is very much inspired by Chris Bail's ["Screen-Scraping in R"](https://cbail.github.io/SICSS_Screenscraping_in_R.html). Thanks Chris!

# Collecting Digital Trace Data: Web Scraping

[Web scraping](https://en.wikipedia.org/wiki/Web_scraping) (also sometimes called "screen-scraping") is a method for extracting data from the web. There are many techniques which can be used for web scraping — ranging from requiring human involvement (“human copy-paste”) to fully automated systems. For research questions where you need to visit many webpages, and collect essentially very similar information from each, web scraping can be a great tool.

The typical web scraping program:
<ol>
    <li> Loads the address of a webpage to be scraped from your list of webpages</li>
    <li> Downloads the HTML or XML of that website</li> 
    <li> Extracts any desired information</li>
    <li> Saves that information in a convenient format (e.g. CSV, JSON, etc.)</li>
</ol>

<img src="https://raw.githubusercontent.com/compsocialscience/summer-institute/master/2018/materials/day2-digital-trace-data/screenscraping/rmarkdown/Screen-Scraping.png"></img>
<em>From Chris Bail's "Screen-Scraping in R": <a href="https://cbail.github.io/SICSS_Screenscraping_in_R.html">https://cbail.github.io/SICSS_Screenscraping_in_R.html</a></em>

## Legality & Politeness

When the internet was young, web scraping was a common and legally acceptable practice for collecting data on the web. But with the rise of online platforms, some of which rely heavily on user-created content (e.g. [Craigslist](https://en.wikipedia.org/wiki/Craigslist_Inc._v._3Taps_Inc.#Opinion_of_the_court)), the data made available on these sites has become recognized by their companies as highly valuable. Furthermore, from a website developer's perspective, web crawlers are able request many pages from your site in rapid succession, increasing server loads, and generally being a nuisance. 

Thus many websites, especially large sites (e.g. Yelp, AllRecipes, Instagram, The New York Times, etc.), have now forbidden "crawlers" / "robots" / "spiders" from harvesting their data in their "Terms of Service" (TOS). From Yelp's <a href="https://www.yelp.com/static?p=tos">Terms of Service</a>:

<img src="terms_of_service.png"></img>

Before embarking on a research project that will involve web scraping, it is important to understand the TOS of the site you plan on collecting data from. 

If the site does allow web scraping (and you've consulted your legal professional), many websites have a [`robots.txt`](https://www.colorado.edu/robots.txt) file that tells search engines and web scrapers, written by researchers like you, how to interact with the site "politely" (i.e. the number of requests that can be made, pages to avoid, etc.).

## Requesting a Webpage in Python

When you visit a webpage, your web browser renders an [HTML document](https://en.wikipedia.org/wiki/HTML) with [CSS](https://en.wikipedia.org/wiki/Cascading_Style_Sheets) and [Javascript](https://en.wikipedia.org/wiki/JavaScript) to produce a visually appealing page. For example, to us, the [Boulder Humane Society's listing of dogs available for adoption](https://www.boulderhumane.org/animals/adoption/dogs) looks something like what's displayed at the top of the browser below:

<img src="boulder_dogs_html.png"></img>

But to your web browser, the page actually looks like the HTML source code (basically instructions for what text and images to show and how to do so) shown at the bottom of the page. To see the source code of a webpage, in Safari, go to `Develop > Show Page Source` or in Chrome, go to `Developer > View Source`.

To request the HTML for a page in Python, you can use the Python package [`requests`](http://docs.python-requests.org/en/master/), as such:

In [3]:
pet_pages = ["https://www.boulderhumane.org/animals/adoption/dogs", 
             "https://www.boulderhumane.org/animals/adoption/cats", 
             "https://www.boulderhumane.org/animals/adoption/adopt_other"]

r = requests.get(pet_pages[0])
html = r.text
print(html[:500]) # Print the first 500 characters of the HTML. Notice how it's the same as the screenshot above.

<!DOCTYPE html>
<head>
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta charset="utf-8" />
<link rel="shortcut icon" href="https://www.boulderhumane.org/sites/default/files/favicon.ico" type="image/vnd.microsoft.icon" />
<meta name="Generator" content="Drupal 7 (http://drupal.org)" />
<meta name="viewport" content="width=1000px, initial-scale=1.0, maximum-scale=1.0" />
<title>Dogs Available for Adoption | Humane Society of Boulder Valley</title>
<link type="text/css" rel="stylesheet


## Parsing HTML with BeautifulSoup

Now that we've downloaded the HTML of the page, we next need to parse it. Let's say we want to extract all of the names, ages, and breeds of the dogs, cats, and small animals currently up for adoption at the Boulder Humane Society. 

Actually, navigating to the location of those attributes in the page can be somewhat tricky. Luckily HTML has a tree-structure, as shown below, where tags fit inside other tags. For example, the `title` of the document is located within its `head`, and within the larger `html` document (`<html> <head> <title> </title> ... </head>... </html>`).

<img src="https://raw.githubusercontent.com/compsocialscience/summer-institute/master/2018/materials/day2-digital-trace-data/screenscraping/rmarkdown/html_tree.png"></img>
<em>From Chris Bail's "Screen-Scraping in R": <a href="https://cbail.github.io/SICSS_Screenscraping_in_R.html">https://cbail.github.io/SICSS_Screenscraping_in_R.html</a></em>

To find the first pet on the page, we'll find that HTML element's "CSS selector". Within Safari, hover your mouse over the image of the first pet and then control+click on the image. This should highlight the section of HTML where the object you are trying to parse is found. Sometimes you may need to move your mouse through the HTML to find the exact location of the object (see [GIF](https://github.com/allisonmorgan/sicss_boulder/blob/master/day_2/copy_selector.gif)).

<img src="copy_selector.png"></img>

(You can also go to 'Develop > Show Page Source' and then click 'Elements'. Hover your mouse over sections of the HTML until the object you are trying to find is highlighted within your browser.) 

[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) is a Python library for parsing HTML. We'll pass the CSS selector that we just copied to BeautifulSoup to grab that object. Notice below how `select`-ing on that pet, shows us all of its attributes.

In [4]:
soup = BeautifulSoup(html, 'html.parser')
pet = soup.select("#block-system-main > div > div > div.view-content > div.views-row.views-row-1.views-row-odd.views-row-first.Adopt.Me")
print(pet)

[<div class="views-row views-row-1 views-row-odd views-row-first Adopt Me">
<div class="views-field views-field-field-pp-photo"> <div class="field-content animal-photo"><a href="/animals/adoption/38897934"><img border="0" height="162" src="https://g.petango.com/photos/993/38fca025-4171-4f09-8593-6a083e90d3d1.jpg" title="Adopt Me" width="216"/></a></div> </div>
<div class="views-field views-field-field-pp-splashtitle"> <div class="field-content">Adopt Me</div> </div>
<div class="views-field views-field-field-pp-animalname"> <div class="field-content"><a href="/animals/adoption/38897934" title="Adopt Me!">Max</a></div> </div>
<div class="views-field views-field-field-pp-primarybreed"> <div class="field-content">Terrier, American Pit Bull</div> </div>
<div class="views-field views-field-field-pp-secondarybreed"> <div class="field-content"></div> </div>
<div class="views-field views-field-field-pp-age"> <span class="views-label views-label-field-pp-age">Age: </span> <span class="field-cont

Furthermore, we can select the name, breeds, age, and gender of the pet by `find`-ing the `div` tags which contain this information. Notice how the `div` tag has the attribute (`attrs`) `class` with value "views-field views-field-field-pp-animalname".

In [5]:
name = pet[0].find('div', attrs = {'class': 'views-field views-field-field-pp-animalname'})
primary_breed = pet[0].find('div', attrs = {'class': 'views-field views-field-field-pp-primarybreed'})
secondary_breed = pet[0].find('div', attrs = {'class': 'views-field views-field-field-pp-secondarybreed'})
age = pet[0].find('div', attrs = {'class': 'views-field views-field-field-pp-age'})

In [6]:
# We can call `get_text()` on those objects to print them nicely.
print({
    "name": name.get_text(strip = True), 
    "primary_breed": primary_breed.get_text(strip = True), 
    "secondary_breed": secondary_breed.get_text(strip = True),
    "age": age.get_text(strip=True)
})

{'name': 'Max', 'primary_breed': 'Terrier, American Pit Bull', 'secondary_breed': '', 'age': 'Age:6 years 11 months'}


Now to get at the HTML object for each pet, we could find the CSS selector for each. Or, we can exploit the fact that every pet lives in a similar  HTML structure for each pet. That is, we can find all of the div tags with the class attribute which contain the string `views-row`. We'll print out their attributes like we just did.

In [7]:
all_pets = soup.find_all('div', {'class': 'views-row'})

In [8]:
for pet in all_pets:
    name = pet.find('div', {'class': 'views-field views-field-field-pp-animalname'}).get_text(strip=True)
    primary_breed = pet.find('div', {'class': 'views-field views-field-field-pp-primarybreed'}).get_text(strip=True)
    secondary_breed = pet.find('div', {'class': 'views-field views-field-field-pp-secondarybreed'}).get_text(strip=True)
    age = pet.find('div', {'class': 'views-field views-field-field-pp-age'}).get_text(strip=True)
    print([name, primary_breed, secondary_breed, age])

['Max', 'Terrier, American Pit Bull', '', 'Age:6 years 11 months']
['Angel', 'Terrier, American Pit Bull', 'Mix', 'Age:3 years 3 months']
['Bruno', 'Terrier, American Pit Bull', 'Mix', 'Age:6 years 6 months']
['Zeus', 'Boxer', 'Mix', 'Age:2 years 1 month']
['Pitkin', 'Retriever, Labrador', 'Mix', 'Age:3 years 0 months']
['Penny', 'Australian Cattle Dog', 'Boxer', 'Age:7 years 0 months']
['Cocoa', 'Bulldog, American', 'Mix', 'Age:0 years 3 months']
['Mona', 'Rottweiler', 'Mix', 'Age:4 years 0 months']
['Little Bit', 'Terrier, Jack Russell', 'Mix', 'Age:7 years 0 months']
['Jefferson', 'Retriever, Chesapeake Bay', 'Mix', 'Age:4 years 0 months']
['Linus', 'Terrier, Airedale', 'Mix', 'Age:8 years 0 months']
['Willow Moon', 'Terrier, American Pit Bull', 'Border Collie', 'Age:1 year 3 months']
['Kenna', 'Boxer', 'Mix', 'Age:7 years 4 months']
['Reginald', 'Terrier, Rat', 'Mix', 'Age:3 years 0 months']
['Christine', 'Chihuahua, Short Coat', 'Mix', 'Age:7 years 0 months']
['Bubba', 'Siberian H

This may seem like a fairly silly example of webscraping, but one could imagine several research questions using this data. For example, if we collected this data over time (e.g. using [Wayback Machine](https://archive.org/web/)), could we identify what features of pets -- names, breeds, ages -- make them more likely to be adopted? Are there certain names that are more common for certain breeds? Or maybe your research question is [something even wackier](https://qz.com/1025049/a-computer-used-artificial-intelligence-to-create-new-dog-names/).

### Aside: Read Tables from Webpages

[Pandas](https://pandas.pydata.org) has really neat functionality in `read_html` where you can download an HTML table directly from a webpage, and load it into a dataframe.

In [9]:
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_sandwiches", header=0)[0]
#table.to_csv("filenamehere.csv") # Write table to CSV

table.head(20)

,Name,Image,Origin,Description
0,Bacon,NaN,United Kingdom,Often eaten with ketchup or brown sauce
1,"Bacon, egg and cheese",NaN,United States,"Breakfast sandwich, usually with fried or scra..."
2,Bagel toast,NaN,Israel,"Pressed, toasted bagel filled with vegetables ..."
3,Baked bean,NaN,United States,"Canned baked beans on white or brown bread, so..."
4,Bánh mì[4],NaN,Vietnam,"Filling is typically meat, but can contain a w..."
5,Barbecue[5][6][7],NaN,United States,"Served on a bun, with chopped, sliced, or shre..."
6,Barros Jarpa,NaN,Chile,"Ham and cheese, usually mantecoso, which is si..."
7,Barros Luco,NaN,Chile,Beef (usually thin-cut steak) and cheese
8,Bauru,NaN,Brazil,"Melted cheese, roast beef, tomato, and pickled..."
9,Beef on weck,NaN,"United States(Buffalo, New York)",Roast beef on a Kummelweck roll


# Requesting a Webpage with Selenium

Sometimes our interactions with webpages involve rendering Javascript. For example, think of visiting a webpage with a search box, typing in a query, pressing search, and viewing the result. Or visiting a webpage that requires a login, or clicking through pages in a list. To handle pages like these we'll use a package in Python called [Selenium](http://selenium-python.readthedocs.io/index.html). (Installing Selenium can be a little tricky. You'll want to follow the directions as best you can [here](http://selenium-python.readthedocs.io/installation.html).)

First a fairly simple example: let's visit [xkcd](https://xkcd.com) and click through the comics.

In [10]:
driver = selenium.webdriver.Safari() # This command opens a window in Safari
# driver = selenium.webdriver.Chrome() # This command opens a window in Chrome

# Get the xkcd website
driver.get("https://xkcd.com/")

In [11]:
# Let's find the 'random' buttom
element = driver.find_element_by_xpath('//*[@id="middleContainer"]/ul[1]/li[3]/a')
element.click()

In [12]:
# Find an attribute of this page - the title of the comic.
element = driver.find_element_by_xpath('//*[@id="comic"]/img')
element.get_attribute("title")

"Eno's storied aria was once soloed by Judge Lance Ito on the alto oboe at Ohio's AirAsia Arena."

In [13]:
# Continue clicking throught the comics
driver.find_element_by_xpath('//*[@id="middleContainer"]/ul[1]/li[3]/a').click()

In [14]:
driver.quit() # Always remember to close your browser!

We'll now walk through how we can use Selenium to navigate the website to navigate a open source site called <a href="https://www.boxofficemojo.com">"Box Office Mojo"</a>.

<img src="box_office_mojo.png"></img>

In [15]:
driver = selenium.webdriver.Safari() # This command opens a window in Safari
# driver = selenium.webdriver.Chrome() # This command opens a window in Chrome

driver.get('https://www.boxofficemojo.com')

Let's say I wanted to know which movie was has been more lucrative 'Wonder Woman', 'Blank Panther', or 'Avengers: Infinity War'. I could type into the search bar on the upper left: 'Avengers: Infinity War'.

In [16]:
# Type in the search bar, and click 'Search'
driver.find_element_by_xpath('//*[@id="leftnav"]/li[2]/form/input[1]').send_keys('Avengers: Infinity War')
driver.find_element_by_xpath('//*[@id="leftnav"]/li[2]/form/input[2]').click()

Now, I can parse the table returned using `BeautifulSoup`.

In [17]:
table = driver.find_element_by_xpath('//*[@id="body"]/table[2]')
table.get_attribute('innerHTML').strip()

NoSuchElementException: Message: An element could not be located on the page using the given search parameters.


In [ ]:
pd.read_html(table.get_attribute('innerHTML').strip(), header=0)[2]

In [ ]:
# Find the link on the page and click it
driver.find_element_by_xpath('//*[@id="body"]/table[2]/tbody/tr/td/table[2]/tbody/tr[2]/td[1]/b/font/a').click()

Now, we can do the same for the remaining movies: 'Wonder Woman', and 'Black Panther' ...

In [ ]:
driver.quit() # Always remember to close your browser!

Again, this might seem like a fairly simple example of web scraping, but there are some fun data science questions you could answer with this new technique. For example, for how many movie franchises, were their sequels more successful than their originals. For example, <a href="https://www.boxofficemojo.com/franchises/chart/?view=main&id=fastandthefurious.htm&sort=gross&order=DESC&p=.htm">Furious 7</a> (see 'Adjusted for Ticket Price Inflation') was the most lucrative <a href="https://en.wikipedia.org/wiki/The_Fast_and_the_Furious">Fast and the Furious</a> movie. 

## Key Takeaways

These two examples hopefully show you how fun web scraping can be, but as I hinted at earlier in some cases web scraping is illegal. So when should you use this new tool? Here are some pointers:

1. When it is not illegal.
2. Ideally, when webpages you are trying to scrape share similar HTML structure.
3. In cases where collecting the data by hand would be prohibitively expensive, in time or money.
3. When the website you are trying to scrape has not made a publicly accessible API or dataset.

# Activity

In this notebook, we learned about how to scrape data from the web using Python's packages `requests`, `BeautifulSoup`, and `Selenium`. Some of you have already had experience with web scraping, but for others, this may have been your first time collecting digital trace data. 

This group exercise is designed to find a balance between practicing rudimentary skills (for those of you with little or no experience in this area) to cutting edge techniques (for those of you with extensive expertise in this area). As an added bonus, this exercise not only challenges you to practice your coding skills, but to think about how to ask questions that contribute new knowledge to sociological theory as well.

<ol>
<li>First, for five minutes, independently brainstorm one or two research questions that you believe can be answered using online data sources and web scraping. </li>
<li>Divide yourselves into groups of three or four. Try to join a group with people you haven't worked with yet.</li>
<li>For 10 minutes, work together to identify a research question based on one of the data sources proposed by your group members.</li>
<li>Evaluate the strengths and weaknesses of the data you plan to collect.</li>
<li>Outline a hybrid research design (e.g. an app or a bot) that could be used to address the weaknesses of the data you collected, or otherwise improve your ability to answer the research question.</li>
<li>(If you have time, write code to collect data from each unit of analysis in your sample. See the code below for help.)</li>
</ol>

There is only one requirement: the group member with the least amount of experience coding should be responsible for typing the code into a computer. After 45 minutes, we will share our work with the group. Let us know if you'd like to present your group's potential project. Remember that these daily exercises are a way for you to explore new possible topics and to get to know each other better.

### List of Online Open Source Data & Websites

Here, is a short list of open source data and websites which may be useful for the group activity above. If you have any to add please tell Allie, or better yet, [start an issue or submit a pull request](https://github.com/allisonmorgan/sicss_boulder).
<ul>
    <li><a href="https://docs.google.com/spreadsheets/d/1wZhPLMCHKJvwOkP4juclhjFgqIY8fQFMemwKL2c64vk/edit#gid=0">Data is Plural" Newsletter</a></li>
    <li><a href="http://archive.ics.uci.edu/ml/index.php">UCI Machine Learning Repository</a></li>
    <li><a href="https://icon.colorado.edu/#!/">University of Colorado Network Dataset</a></li> 
    <li><a href="https://snap.stanford.edu/data/">Stanford Network Datasets</a></li>
    <li><a href="https://networkdata.ics.uci.edu/resources.php">UCI Network Datasets</a></li>
    <li><a href="https://aws.amazon.com/datasets/8172056142375670">Google Books nGrams</a></li>
    <li><a href="http://about.reuters.com/researchandstandards/corpus/">Reuters News Corpus</a></li>
    <li><a href="http://www.nltk.org/nltk_data/">NLTK Corpora</a></li>
    <li><a href="https://www.cs.cmu.edu/~./enron/">Enron Emails Dataset</a></li>
    <li><a href="http://snap.stanford.edu/class/cs224w-2012/projects/cs224w-013-final.pdf">Political Blogs 2008</a></li>
    <li><a href="https://aws.amazon.com/public-data-sets/common-crawl/">The Common Crawl</a></li>
    <li><a href="https://meta.wikimedia.org/wiki/Data_dumps">Wikipedia</a></li>
    <li><a href="http://archive.ics.uci.edu/ml/datasets/Amazon+Commerce+reviews+set">Amazon Commerce Reviews Set</a></li>
    <li><a href="http://archive.ics.uci.edu/ml/datasets/NSF+Research+Award+Abstracts+1990-2003">NSF Research Award Abstracts</a></li>
    <li><a href="http://scikit-learn.org/stable/datasets/twenty_newsgroups.html">20 Newsgroups</a></li>
    <li><a href="https://catalog.ldc.upenn.edu/LDC2008T19">New York Times Annotated Corpus</a></li>
    <li><a href="https://code.google.com/p/graphlabapi/downloads/detail?name=daily_kos.tar.bz2&can=2&q=">Daily Kos Blog Posts</a></li>
    <li><a href="http://deepdive.stanford.edu/doc/opendata/">Stanford DeepDive Open Datasets</a></li>
    <li><a href="http://trec.nist.gov/data/tweets/">Tweets 2011</a></li>
    <li><a href="http://www.boards.ie/">Irish Discussion Boards</a></li>
    <li><a href="http://www.cs.cornell.edu/people/pabo/movie-review-data/">Movie Review Data</a></li>
    <li><a href="https://www.yelp.com/academic_dataset">Yelp Dataset</a></li>
    <li><a href="http://www.biomedcentral.com/1471-2105/15/S11/S11">BMC BioInformatics</a></li>
    <li><a href="http://socialcomputing.asu.edu/datasets/Twitter">ASU Twitter Dataset</a></li>
    <li><a href="https://snap.stanford.edu/data/higgs-twitter.html">Higgs Twitter Network Dataset</a></li>
    <li><a href="http://icwsm.org/2013/datasets/datasets/">ICWSM (Various Datasets)</a></li>
    <li><a href="http://labrosa.ee.columbia.edu/millionsong/musixmatch">Million Song Dataset</a></li>
    <li><a href="https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/XPCVEI">EUSpeech</a></li>
    <li><a href="http://humanrightstexts.org/">Human Rights texts</a></li>
    <li><a href="http://textlab.econ.columbia.edu/~jjacobs/marx/">Marx Corpus</a></li>
</ul>